# Generate tiff files to plot with QGIS

### Once generated files then can be plotted using QGIS
Want to plot them with highlighted urban areas, and these urban areas also extracted and plotted bigger as inset images.  
To do this:  
  - Import shpfile with the boundary of the urban area (e.g. Kippax or Garforth)  
  - Right click -> Properties -> Symbology -> (at top) select Inverted Polygons  

In [80]:
catchment_name = 'WykeBeck'
method_name = 'Observed'
variable_name ='Depth'

In [81]:
from matplotlib import cm
from rasterio import plot
from mpl_toolkits.axes_grid1 import make_axes_locatable
from my_functions import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [82]:
watercourses = gpd.read_file('../../../FloodModelling/ExtraData/Watercourses/WatercourseLink.shp')

In [83]:
# Specify strings relating to catchment
if catchment_name == 'LinDyke':
    catchment_name_str = "Resampled.Terrain" 
    minx, miny, maxx, maxy = 437000,  426500,  445500, 434300
    shpfile_name ='CatchmentLinDyke_exported'
    cell_size_in_m2 = 1
    crop_or_not = True    
elif catchment_name == 'WykeBeck':
    catchment_name_str = "Terrain.wykeDEM" 
    minx, miny, maxx, maxy = 430004,  429978, 438660, 440996 
    cell_size_in_m2 = 4
    shpfile_name ='WykeBeckCatchment'
    crop_or_not = False

In [84]:
if method_name == 'Idealised':
    most_extreme_str ='6h_sp_bl_0.9'
    least_extreme_str ="6h_sp_fl_0.1"
elif method_name == 'Observed':
    most_extreme_str ='6h_c5'
    least_extreme_str="6h_c8"

### Read catchment shapefile

In [85]:
model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles_export/'.format(catchment_name, method_name)
catchment_shp = model_directory + "{}.shp".format(shpfile_name)
catchment_gdf = gpd.read_file(catchment_shp)

# For plot showing cells flooded in both, or just one or the other scenario 

### Read in flood data

In [86]:
# Get the results, and trim to the catchment
most_extreme, out_meta  = open_and_clip_to_catchment(model_directory+ f"{most_extreme_str}/{variable_name} (Max).{catchment_name_str}.tif"
                                                      , catchment_gdf, crop_or_not)
least_extreme, out_meta  = open_and_clip_to_catchment(model_directory+ f"{least_extreme_str}/{variable_name} (Max).{catchment_name_str}.tif"
                                                      , catchment_gdf, crop_or_not)

### Make versions where cells with flooding <0.1m are set to np.nan

In [87]:
most_extreme = np.where(most_extreme > 0.1, most_extreme, np.nan)
least_extreme = np.where(least_extreme > 0.1, least_extreme, np.nan)

In [91]:
fp = model_directory + f"CalculatedLayers/{variable_name}_{most_extreme_str}_over_0.1m.tif"
save_array_as_raster(most_extreme, fp, out_meta)

fp = model_directory + f"CalculatedLayers/{variable_name}_{least_extreme_str}_over_0.1m.tif"
save_array_as_raster(least_extreme, fp, out_meta)

### Make version where cells with flooding are set to 1, and all others are set to np.nan

In [92]:
most_extreme = np.where(most_extreme > 0.1, 1, np.nan)
least_extreme = np.where(least_extreme > 0.1, 1, np.nan)

### Set any NA values to 0, where the other scenario doesn't have an NaN
This allows the difference to be counted for cells which have an NaN in one but not the other

In [93]:
most_extreme[np.isnan(most_extreme) & (~np.isnan(least_extreme))] = 0
least_extreme[np.isnan(least_extreme) & (~np.isnan(most_extreme))] = 0

### Label each cell according to whether it has flooding in neither scenario, both scenarios or 1 or the other

In [94]:
# Flooded in neither scenario
flooded_by_scenario = np.where((np.isnan(most_extreme)& np.isnan(least_extreme)), np.nan, 
        # 0 = flooded in both scenarios (over 0.1m)
        np.where(((least_extreme ==1) & (most_extreme ==1)), 0,
            # 1 = flooded in most_extreme, but not least_extreme
            np.where(((most_extreme ==1) & (least_extreme ==0)), 1,
            np.where(((most_extreme ==1) & np.isnan(least_extreme)), 1,             
                # 2 = flooded in least_extreme, but not most_extreme
                np.where(((least_extreme ==1) & (most_extreme ==0)), 2 ,
                np.where(((least_extreme ==1) & np.isnan(most_extreme)), 2 ,                 
                    # 500 = any other cells (shouldn't be any)        
                     600))))))

In [95]:
fp = model_directory + f"CalculatedLayers/{variable_name}_{method_name}_Flooded_by_scenario.tif"
save_array_as_raster(flooded_by_scenario, fp, out_meta)

# For plot showing cells flooded in both, or just one or the other scenario 

### Read in flood data

In [96]:
# Get the results, and trim to the catchment
most_extreme, out_meta  = open_and_clip_to_catchment(model_directory+ f"{most_extreme_str}/{variable_name} (Max).{catchment_name_str}.tif"
                                                      , catchment_gdf, crop_or_not)
least_extreme, out_meta  = open_and_clip_to_catchment(model_directory+ f"{least_extreme_str}/{variable_name} (Max).{catchment_name_str}.tif"
                                                      , catchment_gdf, crop_or_not)

### Set values <0.1 to np.nan

In [97]:
most_extreme = np.where(most_extreme > 0.1, most_extreme, np.nan)
least_extreme = np.where(least_extreme > 0.1, least_extreme, np.nan)

### Set any NA values to 0, where the other scenario doesn't have an NaN
This allows the difference to be counted for cells which have an NaN in one but not the other

In [98]:
most_extreme[np.isnan(most_extreme) & (~np.isnan(least_extreme))] = 0
least_extreme[np.isnan(least_extreme) & (~np.isnan(most_extreme))] = 0

In [99]:
diff = most_extreme - least_extreme

In [100]:
pos_neg_depth_diff = np.where(np.isnan(diff), np.nan, 
                                  np.where(diff>0,1,
                                       np.where(diff <0,2,0)))


In [101]:
fp = model_directory + f"CalculatedLayers/{variable_name}_diff_over_0.1m.tif"
save_array_as_raster(diff, fp, out_meta)

In [102]:
fp = model_directory + f"CalculatedLayers/{variable_name}_posnegdepthdiff_over_0.1m.tif"
save_array_as_raster(pos_neg_depth_diff, fp, out_meta)